In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import pandas as pd
import random
import operator
pd.options.display.max_rows = None

In [2]:
#definimos la clase board
class board():
    def __init__(self,n=10):
        self.n=n#dimensiones de la cuadricula nxn
        self.current_state=(0,0)#posicion inicial del agente
        self.reward=0#recompensa obtenida por el agente
        self.grid=np.zeros((10,10)) #almacenamos el gridworld como una matriz 3x7
        self.__values=np.zeros((10,10))#creamos una matriz con las mismas dimensiones que el grod world
        self.grid[2,1:5]=np.nan
        self.grid[2,6:9]=np.nan
        self.grid[2:8,4]=np.nan
        #para almacenar los valores de los estados(entiendase valores como los que se calculan de la ecuación de 
        #Bellman)
        self.__values[4,5]=-1
        self.__values[5,5]=1
        self.__values[7,5]=-1
        self.__values[7,6]=-1
        
        
        
        
        self._is_term=np.zeros((10,10))
        self._is_term[4,5]=1 #llenamos las recompensas de la fila superior
        self._is_term[5,5]=1 #llenamos las recompensas de la fila superior
        self._is_term[7,5]=1#el inicio del puente
        self._is_term[7,6]=1#el final del puente
    def get_current_state(self):
        return self.current_state
    def get_posible_actions(self, estado_actual):
        fila, columna=estado_actual #desempacamos la fila y la columna
        n_filas,n_columnas=self.grid.shape
        #del estado actual
        acciones=[]
        n=self.n
        if self.is_terminal(estado_actual)==True:
            acciones.append("Salir")
            return acciones
        #verificar si up es posible:
        if (fila!=0) and (np.isnan(self.grid[fila-1,columna])==False):#se puede mover hacia arriba
            #si no esta en la fila cero y ademas en la casilla superior no hay obstaculo
            acciones.append("up")
        #verificar si down es posible
        if (fila!=n_filas-1) and (np.isnan(self.grid[fila+1,columna])==False):#se puede mover hacia abajo
            #si no esta en la ultima fila y ademas la casilla de abajo no es obstaculo
            acciones.append("down")
        #verificar si right es posible
        if (columna!=n_columnas-1) and (np.isnan(self.grid[fila,columna+1])==False):#se puede mover hacia la
            #derecha si no esta en la ultima columna y ademas la casilla de la derecha no es obstaculo
            acciones.append("right")
        if (columna!=0) and (np.isnan(self.grid[fila,columna-1])==False):#se puede mover hacia la izquierda
            #si no esta en la primera coulumna y ademas la casilla de la izquiera no es un obstaculo
            acciones.append("left")
        return acciones
    
    def do_action(self, action):
        fila_actual, columna_actual=self.current_state
        acciones_posibles=self.get_posible_actions((fila_actual,columna_actual))
        if action not in acciones_posibles:
            return (0,self.current_state)
        
        
        
        if action=="Salir":
            return(self.__values[fila_actual,columna_actual],"END")
              
        if action=="up":#moverse hacia arriba, restar 1 al valor de la fila actual
            self.current_state=(fila_actual-1,columna_actual)
        elif action=="down":#moverse hacia arriba, sumar 1 al valor de la fila actual
            self.current_state=(fila_actual+1,columna_actual)
        elif action=="right":#moverse hacia derecha, sumar 1 al valor de la columna actual
            self.current_state=(fila_actual,columna_actual+1)
        else:
            #moverse hacia izquierda, restar 1 al valor de la columna actual
            self.current_state=(fila_actual,columna_actual-1)
        return (self.grid[self.current_state[0],self.current_state[1]],self.current_state)   
    def reset(self):#reiniciar
        self.current_state=(0,0)
        self.reward=0
    def is_terminal(self,estado):
        if (self._is_term[estado[0],estado[1]]==1) | (self._is_term[estado[0],estado[1]]==-1):
            return True
        else:
            return False
        
    #HASTA AQUI EL PUNTO I) DE LA TAREA, QUE CORRESPONDIA A LA DEFINICION DEL AMBIENTE.
    #PARA LA PARTE II) DE LA TAREA, QUE ES CONSTRUIR EL MARKOV DECISSION PROCESS, DEFINIMOS OTRO
    #METODO, QUE LLAMÉ MDP
    


        
            

In [3]:
class Q_LEARNING():
    def __init__(self,env,epsilon,gamma,alpha):
        self.epsilon=epsilon
        self.gamma=gamma
        self.alpha=alpha
        self.env=env
        #creamos el atributo Q para guardar los valores de las acciones y los estados
        #Inicalizalizamos un diccionario para almacenar los q_valores de las acciones y estados:
        self.Q={}
        filas,columnas=self.env.grid.shape
        acciones=["up","right","left","down","Salir"]
        for i in range(filas):
            for j in range(columnas):
                if np.isnan(self.env.grid[i,j])==False:
                    if self.env.is_terminal((i,j))==True:
                        self.Q[(i,j)]={"Salir":0}
                    else:
                        self.Q[(i,j)]={accion:0 for accion in acciones}
                    
        #inicializamos una lista con todos los posibles estados:
        self.posibles_estados=[]
        for i in range(filas):
            for j in range(columnas):
                if np.isnan(self.env.grid[i,j])==False:
                    self.posibles_estados.append((i,j))
        
        
        
    def choose_action(self,estado):
        #seleccionamos la accion maxima
        accion= max(self.Q[estado], key=self.Q[estado].get)
        #seleccionamos la lista de todas las posibles acciones:
        posibles_acciones=list(self.Q[estado].keys())
        #creamos una lista con las posibles acciones diferentes a la greedy:
        posibles_acciones_diferentes=[]
        for element in posibles_acciones:
            if element!=accion:
                posibles_acciones_diferentes.append(element)
        #ahora escogemos entre seguir la greedy o no:
        
        decision=np.random.choice([0,1],p=[1-self.epsilon,self.epsilon])#0 si sigue la politica greedy, 1 si no
        if decision==0:
            return accion#si salio seguir la greedy, devuelve la accion greedy
        else:
            try:
                return random.choice(posibles_acciones_diferentes)#si salio la opcion de explorar, devuelve una accion 
            except IndexError:
                return accion
                #aleatoria diferente a la greedy
    def Q_max(self,estado):#devuelve el valor del maximo estado al que se puede llegar desde el estado dado:
        accion= max(self.Q[estado], key=self.Q[estado].get)
        return self.Q[estado][accion]
        
                
    def action_function(self,estado1, accion1, recompensa, estado2):
        #implementa el calculo del q_valor de la accion segun el agoritmo de Q_learning
        
        if estado2=="END":
            self.Q[estado1][accion1]=self.Q[estado1][accion1]+self.alpha*(recompensa-
                                                                     self.Q[estado1][accion1])
        else:
            Q_max=self.Q_max(estado2)
            self.Q[estado1][accion1]=self.Q[estado1][accion1]+self.alpha*(recompensa+self.gamma*Q_max-self.Q[estado1][accion1])
    def episodio(self):
        #inicializar un estado S:
        S_1=random.choice(self.posibles_estados)
        
        terminado=False#creamos una variable para saber si el episodio esta terminado
        while terminado==False:
            
            if self.env.is_terminal(S_1)==True:
                #print(S_1)
                recompensa,S_2=self.env.do_action("Salir")#ejecutamos la accion 1 desde el estado 1, y obtenemos la recompensa
                #print("Salir")
                #print(recompensa)
                self.action_function(S_1,"Salir",recompensa,S_2)
                terminado=True
            else:
                           
                #print(S_1)
                accion1=self.choose_action(S_1)#calculamos la accion a tomar para estado S1
                #print(accion1)
                self.env.current_state=S_1
                recompensa,S_2=self.env.do_action(accion1)#ejecutamos la accion 1 desde el estado 1, y obtenemos la recompensa
                #print(recompensa)
                #y quedamos en el estado 2
                
                #finalmente, ejecutamos la funcion action_function:
                self.action_function(S_1,accion1,recompensa,S_2)
                #ahora cambiamos S1 a S2 para el siguiente step

                S_1=S_2
                #verficamos si es terminal
                if accion1=="Salir":
                    terminado=True
    def politica_optima(self,episodios):
        #ejecuta los episodios:
        for i in range(episodios):
            self.episodio()#ejecuta la funcion episodio
            print(f"episodio {i}")
        #ahora calculamos la politica optima
        filas,columnas=self.env.grid.shape#obtenemos el numero de filas y columnas
        decisiones=[]
        self.epsilon=0#ahora solo seguimos politicas greedy
        for i in range(filas):
            lista_fila=[self.choose_action((i,j)) if np.isnan(self.env.grid[i,j])==False else None for j in range(columnas) ]
            decisiones.append(lista_fila)
        return pd.DataFrame(decisiones)
                
        
        
        
        
            
        

In [4]:
my_board=board()


In [5]:
my_q_learning=Q_LEARNING(my_board,0.5,0.5,0.1)

In [6]:
my_q_learning.politica_optima(10000)

episodio 0
episodio 1
episodio 2
episodio 3
episodio 4
episodio 5
episodio 6
episodio 7
episodio 8
episodio 9
episodio 10
episodio 11
episodio 12
episodio 13
episodio 14
episodio 15
episodio 16
episodio 17
episodio 18
episodio 19
episodio 20
episodio 21
episodio 22
episodio 23
episodio 24
episodio 25
episodio 26
episodio 27
episodio 28
episodio 29
episodio 30
episodio 31
episodio 32
episodio 33
episodio 34
episodio 35
episodio 36
episodio 37
episodio 38
episodio 39
episodio 40
episodio 41
episodio 42
episodio 43
episodio 44
episodio 45
episodio 46
episodio 47
episodio 48
episodio 49
episodio 50
episodio 51
episodio 52
episodio 53
episodio 54
episodio 55
episodio 56
episodio 57
episodio 58
episodio 59
episodio 60
episodio 61
episodio 62
episodio 63
episodio 64
episodio 65
episodio 66
episodio 67
episodio 68
episodio 69
episodio 70
episodio 71
episodio 72
episodio 73
episodio 74
episodio 75
episodio 76
episodio 77
episodio 78
episodio 79
episodio 80
episodio 81
episodio 82
episodio 83
ep

episodio 714
episodio 715
episodio 716
episodio 717
episodio 718
episodio 719
episodio 720
episodio 721
episodio 722
episodio 723
episodio 724
episodio 725
episodio 726
episodio 727
episodio 728
episodio 729
episodio 730
episodio 731
episodio 732
episodio 733
episodio 734
episodio 735
episodio 736
episodio 737
episodio 738
episodio 739
episodio 740
episodio 741
episodio 742
episodio 743
episodio 744
episodio 745
episodio 746
episodio 747
episodio 748
episodio 749
episodio 750
episodio 751
episodio 752
episodio 753
episodio 754
episodio 755
episodio 756
episodio 757
episodio 758
episodio 759
episodio 760
episodio 761
episodio 762
episodio 763
episodio 764
episodio 765
episodio 766
episodio 767
episodio 768
episodio 769
episodio 770
episodio 771
episodio 772
episodio 773
episodio 774
episodio 775
episodio 776
episodio 777
episodio 778
episodio 779
episodio 780
episodio 781
episodio 782
episodio 783
episodio 784
episodio 785
episodio 786
episodio 787
episodio 788
episodio 789
episodio 790

episodio 1440
episodio 1441
episodio 1442
episodio 1443
episodio 1444
episodio 1445
episodio 1446
episodio 1447
episodio 1448
episodio 1449
episodio 1450
episodio 1451
episodio 1452
episodio 1453
episodio 1454
episodio 1455
episodio 1456
episodio 1457
episodio 1458
episodio 1459
episodio 1460
episodio 1461
episodio 1462
episodio 1463
episodio 1464
episodio 1465
episodio 1466
episodio 1467
episodio 1468
episodio 1469
episodio 1470
episodio 1471
episodio 1472
episodio 1473
episodio 1474
episodio 1475
episodio 1476
episodio 1477
episodio 1478
episodio 1479
episodio 1480
episodio 1481
episodio 1482
episodio 1483
episodio 1484
episodio 1485
episodio 1486
episodio 1487
episodio 1488
episodio 1489
episodio 1490
episodio 1491
episodio 1492
episodio 1493
episodio 1494
episodio 1495
episodio 1496
episodio 1497
episodio 1498
episodio 1499
episodio 1500
episodio 1501
episodio 1502
episodio 1503
episodio 1504
episodio 1505
episodio 1506
episodio 1507
episodio 1508
episodio 1509
episodio 1510
episod

episodio 2109
episodio 2110
episodio 2111
episodio 2112
episodio 2113
episodio 2114
episodio 2115
episodio 2116
episodio 2117
episodio 2118
episodio 2119
episodio 2120
episodio 2121
episodio 2122
episodio 2123
episodio 2124
episodio 2125
episodio 2126
episodio 2127
episodio 2128
episodio 2129
episodio 2130
episodio 2131
episodio 2132
episodio 2133
episodio 2134
episodio 2135
episodio 2136
episodio 2137
episodio 2138
episodio 2139
episodio 2140
episodio 2141
episodio 2142
episodio 2143
episodio 2144
episodio 2145
episodio 2146
episodio 2147
episodio 2148
episodio 2149
episodio 2150
episodio 2151
episodio 2152
episodio 2153
episodio 2154
episodio 2155
episodio 2156
episodio 2157
episodio 2158
episodio 2159
episodio 2160
episodio 2161
episodio 2162
episodio 2163
episodio 2164
episodio 2165
episodio 2166
episodio 2167
episodio 2168
episodio 2169
episodio 2170
episodio 2171
episodio 2172
episodio 2173
episodio 2174
episodio 2175
episodio 2176
episodio 2177
episodio 2178
episodio 2179
episod

episodio 2749
episodio 2750
episodio 2751
episodio 2752
episodio 2753
episodio 2754
episodio 2755
episodio 2756
episodio 2757
episodio 2758
episodio 2759
episodio 2760
episodio 2761
episodio 2762
episodio 2763
episodio 2764
episodio 2765
episodio 2766
episodio 2767
episodio 2768
episodio 2769
episodio 2770
episodio 2771
episodio 2772
episodio 2773
episodio 2774
episodio 2775
episodio 2776
episodio 2777
episodio 2778
episodio 2779
episodio 2780
episodio 2781
episodio 2782
episodio 2783
episodio 2784
episodio 2785
episodio 2786
episodio 2787
episodio 2788
episodio 2789
episodio 2790
episodio 2791
episodio 2792
episodio 2793
episodio 2794
episodio 2795
episodio 2796
episodio 2797
episodio 2798
episodio 2799
episodio 2800
episodio 2801
episodio 2802
episodio 2803
episodio 2804
episodio 2805
episodio 2806
episodio 2807
episodio 2808
episodio 2809
episodio 2810
episodio 2811
episodio 2812
episodio 2813
episodio 2814
episodio 2815
episodio 2816
episodio 2817
episodio 2818
episodio 2819
episod

...

episodio 7915
episodio 7916
episodio 7917
episodio 7918
episodio 7919
episodio 7920
episodio 7921
episodio 7922
episodio 7923
episodio 7924
episodio 7925
episodio 7926
episodio 7927
episodio 7928
episodio 7929
episodio 7930
episodio 7931
episodio 7932
episodio 7933
episodio 7934
episodio 7935
episodio 7936
episodio 7937
episodio 7938
episodio 7939
episodio 7940
episodio 7941
episodio 7942
episodio 7943
episodio 7944
episodio 7945
episodio 7946
episodio 7947
episodio 7948
episodio 7949
episodio 7950
episodio 7951
episodio 7952
episodio 7953
episodio 7954
episodio 7955
episodio 7956
episodio 7957
episodio 7958
episodio 7959
episodio 7960
episodio 7961
episodio 7962
episodio 7963
episodio 7964
episodio 7965
episodio 7966
episodio 7967
episodio 7968
episodio 7969
episodio 7970
episodio 7971
episodio 7972
episodio 7973
episodio 7974
episodio 7975
episodio 7976
episodio 7977
episodio 7978
episodio 7979
episodio 7980
episodio 7981
episodio 7982
episodio 7983
episodio 7984
episodio 7985
episod

episodio 8571
episodio 8572
episodio 8573
episodio 8574
episodio 8575
episodio 8576
episodio 8577
episodio 8578
episodio 8579
episodio 8580
episodio 8581
episodio 8582
episodio 8583
episodio 8584
episodio 8585
episodio 8586
episodio 8587
episodio 8588
episodio 8589
episodio 8590
episodio 8591
episodio 8592
episodio 8593
episodio 8594
episodio 8595
episodio 8596
episodio 8597
episodio 8598
episodio 8599
episodio 8600
episodio 8601
episodio 8602
episodio 8603
episodio 8604
episodio 8605
episodio 8606
episodio 8607
episodio 8608
episodio 8609
episodio 8610
episodio 8611
episodio 8612
episodio 8613
episodio 8614
episodio 8615
episodio 8616
episodio 8617
episodio 8618
episodio 8619
episodio 8620
episodio 8621
episodio 8622
episodio 8623
episodio 8624
episodio 8625
episodio 8626
episodio 8627
episodio 8628
episodio 8629
episodio 8630
episodio 8631
episodio 8632
episodio 8633
episodio 8634
episodio 8635
episodio 8636
episodio 8637
episodio 8638
episodio 8639
episodio 8640
episodio 8641
episod

episodio 9185
episodio 9186
episodio 9187
episodio 9188
episodio 9189
episodio 9190
episodio 9191
episodio 9192
episodio 9193
episodio 9194
episodio 9195
episodio 9196
episodio 9197
episodio 9198
episodio 9199
episodio 9200
episodio 9201
episodio 9202
episodio 9203
episodio 9204
episodio 9205
episodio 9206
episodio 9207
episodio 9208
episodio 9209
episodio 9210
episodio 9211
episodio 9212
episodio 9213
episodio 9214
episodio 9215
episodio 9216
episodio 9217
episodio 9218
episodio 9219
episodio 9220
episodio 9221
episodio 9222
episodio 9223
episodio 9224
episodio 9225
episodio 9226
episodio 9227
episodio 9228
episodio 9229
episodio 9230
episodio 9231
episodio 9232
episodio 9233
episodio 9234
episodio 9235
episodio 9236
episodio 9237
episodio 9238
episodio 9239
episodio 9240
episodio 9241
episodio 9242
episodio 9243
episodio 9244
episodio 9245
episodio 9246
episodio 9247
episodio 9248
episodio 9249
episodio 9250
episodio 9251
episodio 9252
episodio 9253
episodio 9254
episodio 9255
episod

episodio 9834
episodio 9835
episodio 9836
episodio 9837
episodio 9838
episodio 9839
episodio 9840
episodio 9841
episodio 9842
episodio 9843
episodio 9844
episodio 9845
episodio 9846
episodio 9847
episodio 9848
episodio 9849
episodio 9850
episodio 9851
episodio 9852
episodio 9853
episodio 9854
episodio 9855
episodio 9856
episodio 9857
episodio 9858
episodio 9859
episodio 9860
episodio 9861
episodio 9862
episodio 9863
episodio 9864
episodio 9865
episodio 9866
episodio 9867
episodio 9868
episodio 9869
episodio 9870
episodio 9871
episodio 9872
episodio 9873
episodio 9874
episodio 9875
episodio 9876
episodio 9877
episodio 9878
episodio 9879
episodio 9880
episodio 9881
episodio 9882
episodio 9883
episodio 9884
episodio 9885
episodio 9886
episodio 9887
episodio 9888
episodio 9889
episodio 9890
episodio 9891
episodio 9892
episodio 9893
episodio 9894
episodio 9895
episodio 9896
episodio 9897
episodio 9898
episodio 9899
episodio 9900
episodio 9901
episodio 9902
episodio 9903
episodio 9904
episod

0      1      2      3      4      5      6     7      8     9
0  right  right  right   down   down   down   down  left   down  down
1  right  right  right  right  right   down   left  left  right  down
2     up   None   None   None   None   down   None  None   None  down
3     up   left   down   down   None  right   down  down   down  left
4     up  right   down   down   None  Salir   down  down   left  left
5   down   down  right   down   None  Salir   left  left   left  left
6  right  right  right   down   None     up     up    up   left  left
7  right   down  right   down   None  Salir  Salir    up   left    up
8  right  right  right  right  right  right  right    up   left    up
9  right     up  right     up     up     up     up    up   left  left

In [8]:
#LA QTABLA

In [7]:
my_q_learning.Q

{(0, 0): {'up': 9.48957947232869e-05,
  'right': 0.00022018298072055522,
  'left': 9.806380082722474e-05,
  'down': 0.00021157740040293635,
  'Salir': 9.129563802800712e-05},
 (0, 1): {'up': 0.00021237361406328708,
  'right': 0.0004399155115879336,
  'left': 0.00010357431447429443,
  'down': 0.0004372467429794357,
  'Salir': 0.00021797053291094872},
 (0, 2): {'up': 0.00043672377729085047,
  'right': 0.0008780383405352086,
  'left': 0.0002194882536043033,
  'down': 0.0008776932537400849,
  'Salir': 0.00043725495198889026},
 (0, 3): {'up': 0.0008663845984074619,
  'right': 0.0017348642168138178,
  'left': 0.0004350904194538982,
  'down': 0.0017508644375678524,
  'Salir': 0.0008594583970889892},
 (0, 4): {'up': 0.0017385789363417762,
  'right': 0.003480360328061483,
  'left': 0.0008533318637376513,
  'down': 0.003511003548176027,
  'Salir': 0.001713183733548466},
 (0, 5): {'up': 0.0035166762197569503,
  'right': 0.0017160940191386764,
  'left': 0.0017273151540033627,
  'down': 0.007061876